## 1. Configuração Inicial e Instalação de Dependências

In [1]:
!pip install dotenv -q
!pip install sendgrid -q 


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\mathe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\mathe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## 2. Importação de Bibliotecas

In [2]:
import os
from dotenv import load_dotenv
from google.cloud import bigquery
import smtplib
from email.mime.text import MIMEText

## 3. Credenciais e Variáveis de Ambiente

In [3]:
# Célula de Inicialização de Variáveis e Credenciais

# 1. Carrega todas as variáveis do arquivo .env para o ambiente
load_dotenv(r"C:\Users\mathe\Downloads\archive (10)\.env.txt")

# 2. Garante que o caminho da chave de serviço (ADC)

chave_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
if chave_path:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = chave_path
    print("Caminho da chave de serviço configurado via os.environ.")
else:
    print("Atenção: GOOGLE_APPLICATION_CREDENTIALS não foi encontrado no .env!")

Caminho da chave de serviço configurado via os.environ.


In [4]:
# Carregar variáveis do .env
load_dotenv()

# Inicializar cliente BigQuery
client = bigquery.Client()

# Query para buscar tintas próximas ao vencimento
query = """
SELECT
  Loja, Tinta, FORMAT_TIMESTAMP('%d/%m/%Y', Validade) AS Validade,
  dias_ate_expirar, Estoque
FROM `challenge-421602.acoes_recomendadas.levantamento_atual_com_valores`
WHERE dias_ate_expirar < 60
ORDER BY dias_ate_expirar ASC
"""

results = client.query(query).result()

# Construir corpo do e-mail
linhas = []
for row in results:
    linhas.append(
        f"{row['Loja']} - {row['Tinta']} (vence em {row['dias_ate_expirar']} dias - {row['Validade']}) | Estoque: {row['Estoque']}"
    )

if not linhas:
    print("Nenhuma tinta próxima ao vencimento.")
    exit()

corpo_email = "🚨 Tintas próximas ao vencimento:\n\n" + "\n".join(linhas)

# Configuração do e-mail
remetente = os.getenv("GMAIL_USER")
senha = os.getenv("GMAIL_APP_PASSWORD")
destinatario = os.getenv("DESTINATARIO_EMAIL")

msg = MIMEText(corpo_email)
msg["Subject"] = "🚨 Alerta: Tintas próximas ao vencimento"
msg["From"] = remetente
msg["To"] = destinatario

# Enviar via Gmail (SMTP)
with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
    smtp.login(remetente, senha)
    smtp.sendmail(remetente, destinatario, msg.as_string())

print("Email enviado com sucesso!")
print("Remetente:", remetente)
print("Destinatário:", destinatario)
print("Corpo do e-mail:\n", corpo_email)



Email enviado com sucesso!
Remetente: matheussebastiaomendes9@gmail.com
Destinatário: matheus_sebastiaomendes@hotmail.com
Corpo do e-mail:
 🚨 Tintas próximas ao vencimento:

São Paulo - Centro - Tinta Amarela (vence em 26 dias - 01/06/2025) | Estoque: 150
São Paulo - Marginal Tietê - Tinta Brilhante (vence em 26 dias - 01/06/2025) | Estoque: 300
São Paulo - Marginal Tietê - Tinta Anti-Mofo (vence em 26 dias - 01/06/2025) | Estoque: 150
São Paulo - Centro - Tinta Metálica (vence em 26 dias - 01/06/2025) | Estoque: 200
São Paulo - Marginal Tietê - Tinta Azul (vence em 26 dias - 01/06/2025) | Estoque: 200
São Paulo - Marginal Tietê - Tinta Esmalte (vence em 26 dias - 01/06/2025) | Estoque: 600
São Paulo - Centro - Tinta Preta (vence em 29 dias - 07/06/2025) | Estoque: 165
São Paulo - Centro - Tinta Spray (vence em 29 dias - 07/06/2025) | Estoque: 231
São Paulo - Marginal Tietê - Tinta Anti-Mofo (vence em 29 dias - 07/06/2025) | Estoque: 198
São Paulo - Centro - Tinta Amarela (vence em 29 